In [1]:
#Test für den bestmöglichen Lernalgorithmus, durchgeführt mit den einfach bereinigten Daten(160-Set) 


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas  as pd
from io import StringIO
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/preprocessed_data/basic_cleaned_160.csv')

In [4]:
col = ['content', 'author']
data = data[col]
data.columns =['content', 'author']

In [5]:
data['author_id'] = data['author'].factorize()[0]
product_id_data = data[['author', 'author_id']].drop_duplicates().sort_values('author_id')
product_to_id = dict(product_id_data.values)
id_to_product = dict(product_id_data[['author_id', 'author']].values)

In [6]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 2))
features = tfidf.fit_transform(data.content.values.astype('U')).toarray()

In [7]:
labels = data.author_id

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data['content'].values.astype('U'), data['author'].values.astype('U'), random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [9]:
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),#Linear Support Vector Classification.
    MultinomialNB(),#Naive Bayes classifier for multinomial models
    LogisticRegression(random_state=0),
]
CV = 10
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [10]:
#Logistische Regression hat den besten Accuracy-Score

cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 0.531250
LogisticRegression        0.545312
MultinomialNB             0.507812
RandomForestClassifier    0.519531
Name: accuracy, dtype: float64